In [1]:
import os

In [2]:
%pwd

'd:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransormationConfig:
    root_dir: Path
    data_path: Path



In [5]:
from src.OLAChurnPred.constants import *
from src.OLAChurnPred.utils.common import read_yaml, create_directories
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransormationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransormationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path
        )

        return data_transformation_config


In [6]:
import os
from src.OLAChurnPred import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from src.OLAChurnPred.utils.data_transformation_utils import *
from src.OLAChurnPred.utils.common import save_bin
from sklearn.preprocessing import OneHotEncoder

In [7]:
class DataTransormation:
    def __init__(self, config: DataTransormationConfig):
        self.config = config

    def transform(self):
        try:
            data = pd.read_csv(self.config.data_path)
            data = change_data_types(data)
            data = missing_value_imputation(data)
            data = group_transform_data(data)
            return data
        except Exception as e:
            raise e
        
    def train_test_splitting(self, data):
        train,test = train_test_split(data, test_size=0.20)

        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)
        logger.info("Train and test files created")
        print(train.shape, test.shape)    




In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransormation(config=data_transformation_config) 
    data = data_transformation.transform()
    data_transformation.train_test_splitting(data)
except Exception as e:
    raise e

[2024-12-11 23:18:18,221: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-11 23:18:18,221: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-11 23:18:18,221: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-11 23:18:18,221: INFO: common: created directory at: artifacts]
[2024-12-11 23:18:18,237: INFO: common: created directory at: artifacts/data_transformation]


d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Total Business Value'].replace(to_replace=0, value = np.nan, inplace=True)
d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

[2024-12-11 23:18:26,639: INFO: 1554511082: Train and test files created]
(1904, 39) (477, 39)


d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:48: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df1['Age'] = list(df.groupby('Driver_ID',axis=0).max('MMM-YY')['Age'])
d:\Scaler Course\OLACaseStudy\Ola-driver-churn\src\OLAChurnPred\utils\data_transformation_utils.py:55: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df1['Total_Business_Value'] = list(df.groupby('Driver_ID',axis=0).sum('Total Business Value')['Total Business Value'])


In [9]:
df1 = pd.read_csv('D:\\Scaler Course\\OLACaseStudy\\Ola-driver-churn\\artifacts\\data_transformation\\test.csv')
df1.head()

,Age,Gender,Education,Income,Joining_Designation,Grade,Total_Business_Value,Last_Quarterly_Rating,Quarterly_Rating_Increased,Salary_Increased,...,City_C27,City_C28,City_C29,City_C3,City_C4,City_C5,City_C6,City_C7,City_C8,City_C9
0,37.0,0.0,0.0,61243.0,1.0,1.0,5750818.0,3.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,37.0,0.0,1.0,77831.0,1.0,3.0,18472930.0,2.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,39.0,0.0,1.0,29312.0,1.0,1.0,1605888.0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,39.0,1.0,0.0,85693.0,1.0,3.0,4297506.0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,35.0,1.0,1.0,98568.0,2.0,2.0,726504.0,1.0,0,0,...,0,1,0,0,0,0,0,0,0,0
